In [ ]:
import numpy as np

In [ ]:
import PIL.Image
from io import BytesIO
import IPython.display
import numpy as np

In [ ]:
# def array_viz(a, fmt='png'):
#     a = 255 - (a.astype(np.uint8) * 255)
#     f = BytesIO()
#     PIL.Image.fromarray(a).save(f, fmt)
#     return IPython.display.Image(data=f.getvalue())
# def showarray(a):
#     IPython.display.display(array_viz(a))

### Display Formatters

In [ ]:
import html
html_formatter = get_ipython().display_formatter.formatters['text/html']

def bool_array_to_image(a, fmt='png'):
    a = 240 - (a.astype(np.uint8) * 240)
    f = BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    return IPython.display.Image(data=f.getvalue())

def double_bitmap(a):
    (h, w) = a.shape
    return np.tile(a.reshape((-1, 1)), 2).reshape((h, w*2))

def quad_bitmap(a):
    return double_bitmap(double_bitmap(a).T).T

def ndarray_to_html(a):
    if len(a.shape) == 2 and a.max() <= 1 and a.min() >= 0:
        while a.size < 1024:
            a = quad_bitmap(a)
        
        for bundle in bool_array_to_image(a)._repr_mimebundle_():
            for mimetype, b64value in bundle.items():
                if mimetype.startswith('image/'):
                    src = f'data:{mimetype};base64,{b64value}'
                    return f'<img src="{src}">'
    else:
        return "<pre>{}</pre>".format(html.escape(repr(a)))

html_formatter.for_type(np.ndarray, ndarray_to_html)

def item_to_html(item):
    formatter = html_formatter.for_type(type(item))
    if callable(formatter):
        item = formatter(item)

    if callable(getattr(item, '_repr_mimebundle_', None)):
        for bundle in item._repr_mimebundle_():
            for mimetype, b64value in bundle.items():
                if mimetype.startswith('image/'):
                    src = f'data:{mimetype};base64,{b64value}'
                    return f'<img src={src}>'
    
    if not formatter:
        item = '<tt>{}</tt>'.format(html.escape(repr(item)))
    return f'<span style="margin-left:0.2em">{item}</span>'

def list_to_html(l):
    items = [item_to_html(item) for item in l]
    return '<tt style="display:flex;flex-wrap:wrap;align-items:center">[{}]</tt>'.format(', '.join(items))

html_formatter.for_type(list, list_to_html)

def tuple_to_html(l):
    items = (item_to_html(item) for item in l)
    return '<tt style="display:flex;flex-wrap:wrap;align-items:center">({})</tt>'.format(', '.join(items))

html_formatter.for_type(tuple, tuple_to_html)

["Text", np.array([[1,2],[3,4]]), np.eye(10), [(i, np.random.randint(0,2,(10,10))) for i in range(1,4)]]

## Part 1

In [ ]:
tiles = {}
current_tile_id = None
current_tile = []

def make_tile_array(tile_text):
    tile_array = np.array([list(row) for row in tile_text]) == '#'
    return tile_array.astype(int)

with open("input.txt") as f:
    for line in f:
        line = line.strip()
        if line.startswith("Tile "):
            current_tile_id = int(line[4:-1])
            current_tile = []
        elif len(line) == 0:
            tiles[current_tile_id] = make_tile_array(current_tile)
        else:
            current_tile.append(line)

tile_ids = np.array(sorted(list(tiles.keys())))

In [ ]:
[*tiles.values()]

In [ ]:
tile = tiles[tile_ids[0]]
print(tile)

In [ ]:
def normalize_edge(edge):
    a = np.packbits(edge).view(np.uint16)[0]
    b = np.packbits(np.flip(edge)).view(np.uint16)[0]
    return a
[normalize_edge(x) for x in tile]

In [ ]:
def apply_rot(tile, rot_type):
    if rot_type <= 0:
        return tile
    elif rot_type < 4:
        return apply_rot(np.rot90(tile), rot_type-1)
    else:
        return apply_rot(tile.T, rot_type-4)

[apply_rot(tile, i) for i in range(8)]

In [ ]:
def get_edge_sigs(tile):
    edges = [apply_rot(tile, i)[0] for i in range(8)]
    # edges = [ tile[ 0,: ], tile[: ,-1], np.flip(tile[ 0,: ]), np.flip(tile[: ,-1]), tile[-1,: ], tile[: , 0], np.flip(tile[: , 0]), np.flip(tile[-1,: ]) ]
    return np.array([normalize_edge(e) for e in edges])

np.array([get_edge_sigs(apply_rot(tile, i)) for i in range(8)])

In [ ]:
tile_edges = np.array([get_edge_sigs(tiles[tile_id]) for tile_id in tile_ids])
tile_edges[0:5]

In [ ]:
def find_num_matching_borders():
    unique, index, inverse, counts = np.unique(tile_edges, return_index=True, return_inverse=True, return_counts=True)
    matches = counts[inverse].reshape((-1,8))
    return matches
num_border_matches = find_num_matching_borders()
num_border_matches[:5]

In [ ]:
corners = np.sum(num_border_matches, axis=1) == 12
corner_ids = tile_ids[corners]
corner_ids

### Part 1 Solution

In [ ]:
np.product(corner_ids)

## Part 2

In [ ]:
monster = make_tile_array(
    ['                  # ',
     '#    ##    ##    ###',
     ' #  #  #  #  #  #   '])
monster

In [ ]:
def remove_borders(tile):
    return tile[1:-1, 1:-1]
[tile, remove_borders(tile)]

In [ ]:
len(tile_ids)

In [ ]:
picture = np.vstack([np.hstack([remove_borders(tiles[id]) for id in tile_ids[12*i:12*i+12]]) for i in range(12) ])
quad_bitmap(picture)

In [ ]:
get_edge_sigs(apply_rot(tiles[1109], 0))

In [ ]:
np.array([get_edge_sigs(apply_rot(tiles[1181], i)) for i in range(8)])

In [ ]:
rot_lookup_d = [0,1,2,3,4,5,6,7]
def find_next_tile_d(prev_tile_id, prev_rot_type):
    prev_tile = apply_rot(tiles[prev_tile_id], prev_rot_type)
    bot_border = prev_tile[-1:]
    bot_border_sig = normalize_edge(bot_border)
    border_matches, rot_type = np.where(tile_edges == bot_border_sig)
    match_tile_ids = tile_ids[border_matches]
    which = np.where(match_tile_ids != prev_tile_id)
    next_tile_id = match_tile_ids[which][0]
    rot = rot_lookup_d[rot_type[which][0]]
    return next_tile_id, rot
find_next_tile_d(1109, 0)

In [ ]:
def build_col(first_tile_id, first_tile_rot):
    col = [(first_tile_id, first_tile_rot)]
    for _ in range(11):
        prev_tile_id, prev_rot = col[-1]
        next_tile_id, next_rot = find_next_tile_d(prev_tile_id, prev_rot)
        col.append((next_tile_id, next_rot))
#         prev_tile = apply_rot(tiles[prev_tile_id], prev_rot)
#         for i in range(8):
#             next_tile = apply_rot(tiles[next_tile_id], i)
#             if np.array_equal(prev_tile[-1,], next_tile[0,]):
#                 print(next_rot, i)
#                 col.append((next_tile_id, i))
    return col
jigsaw_left = build_col(1109,0)
jigsaw_left

In [ ]:
rot_lookup = [4,7,6,5,0,3,2,1]
def find_next_tile(prev_tile_id, prev_rot_type):
    prev_tile = apply_rot(tiles[prev_tile_id], prev_rot_type)
    right_border = prev_tile[:,-1]
    right_border_sig = normalize_edge(right_border)
    border_matches, rot_type = np.where(tile_edges == right_border_sig)
    match_tile_ids = tile_ids[border_matches]
    which = np.where(match_tile_ids != prev_tile_id)
    next_tile_id = match_tile_ids[which][0]
    rot = rot_lookup[rot_type[which][0]]
    return next_tile_id, rot
find_next_tile(1109, 0)

In [ ]:
def build_row(first_tile_id, first_tile_rot):
    row = [(first_tile_id, first_tile_rot)]
    for _ in range(11):
        prev_tile_id, prev_rot = row[-1]
        next_tile_id, next_rot = find_next_tile(prev_tile_id, prev_rot)
        row.append((next_tile_id, next_rot))
#         prev_tile = apply_rot(tiles[prev_tile_id], prev_rot)
#         for i in range(8):
#             next_tile = apply_rot(tiles[next_tile_id], i)
#             if np.array_equal(prev_tile[:,-1], next_tile[:, 0]):
#                 print(next_rot, i)
#                 row.append((next_tile_id, i))
    return row
jigsaw = [build_row(i,r) for (i,r) in jigsaw_left]
jigsaw[0]

In [ ]:
corner_ids

In [ ]:
def get_inner_tile(tile_id, rot):
    tile = apply_rot(tiles[tile_id], rot)
    return remove_borders(tile)
get_inner_tile(1109, 0)

In [ ]:
picture = np.vstack([
    np.hstack([get_inner_tile(i,r) for (i,r) in jigsaw[r]])
    for r in range(len(jigsaw))
])
print(picture.shape)
quad_bitmap(quad_bitmap(apply_rot(picture,2)))

In [ ]:
with open("seas.png", "wb") as f:
    a = 255 - (apply_rot(picture,2).astype(np.uint8) * 255)
    PIL.Image.fromarray(a).save(f, "png")

In [ ]:
print(monster.shape)
print(np.count_nonzero(monster))
monster

In [ ]:
def find_monster(sea):
    for r in range(8):
        num_found = 0
        p = apply_rot(sea, r)
        for i in range(p.shape[0] - monster.shape[0]):
            for j in range(p.shape[1] - monster.shape[1]):
                crop = p[i:i+monster.shape[0], j:j+monster.shape[1]]
                if np.array_equal(crop & monster, monster):
                    num_found += 1
#                     print("found", r, i, j)
        if num_found > 0:
            return num_found
find_monster(picture)

## Part 2 Solution

In [ ]:
waters = np.count_nonzero(picture) - find_monster(picture) * np.count_nonzero(monster)
waters

In [ ]:
current_tile = []
with open("marked") as f:
    for line in f:
        current_tile.append(line.strip())

marked = make_tile_array(current_tile)
marked

In [ ]:
marked = marked | (np.array([list(row) for row in current_tile]) == 'O').astype(int)
marked

In [ ]:
find_monster(marked)